# Getting Started

MSA (Multi-perturbation Shapley value Analysis) is a Game theoretical approach for calculating the contribution of each element of a system (here network models of the brain) to a system-wide description of the system. The classic neuroscience example: How much each brain region is causally relevant to an arbitrary cognitive function. 

The following examples show some of msapy's capabilities and give you an idea of the API.

For more details on the API, see [API Docs](https://kuffmode.github.io/msa/API/interface.html).

For specific purposes, refer to the following examples:
1. [PyTorch and MSA](https://kuffmode.github.io/msa/examples/mnist_torch.html)
2. [Time Series MSA](https://kuffmode.github.io/msa/examples/MSA%20TimeSeries.html)
3. [MSA 2D](https://kuffmode.github.io/msa/examples/msa2d.html)

To start the code examples, we load the library

In [1]:
import numpy as np

from msapy import msa

Lets assume for this example that we have a system of four elements: "A", "B", "C", "D". The system produces some activity that is equal to the sum of contributions of the individual elements. For the sake of this example, we take the contribution of each element to be 50, 100, 200 and -30 with some added noise. We can write:
* $ A \sim \mathcal{N}(50,\,1)\, $
* $ B \sim \mathcal{N}(100,\,1)\, $
* $ C \sim \mathcal{N}(200,\,1)\, $
* $ D \sim \mathcal{N}(-30,\,1)\, $

In [2]:
nodes = ['A', 'B', 'C', 'D']

Now you need an objective function. An objective function returns the value that you want to know how it's affected by different elements of your system. For this example, it's the total activity of our system. The objective function should take an argument called `complements` which specifies the elements that are leasioned or removed.

In [3]:
def objective_function(complements):
    contributions = {"A": 50, "B": 100, "C": 200, "D": -30}
    activity = 0.0
    for k, v in contributions.items():
        if k not in complements:
            activity += np.random.normal(v, 1)
    return activity

Now that we have all the things to run MSA, we can just call a single function

In [4]:
shapley_table, _, _ = msa.interface(multiprocessing_method='joblib',
                                    elements=nodes,
                                    n_permutations=1000,
                                    objective_function=objective_function,
                                    n_parallel_games=-1,
                                    random_seed=1)

Playing the games: 100%|██████████| 16/16 [00:00<00:00, 55.66it/s]


The `shapley_table` returned from `msa.interface` is a Pandas DataFrame with columns as the elements and the rows as permutations. We can take the mean of the rows to calculate the shapley values i.e. the contribution of each element.

In [5]:
shapley_table.head()

,A,B,C,D
0,50.694676,96.433222,205.187005,-29.750080
1,50.171534,98.596692,203.478361,-29.681764
2,50.171534,101.138584,200.936469,-29.681764
3,53.112195,100.518344,201.291574,-32.357290
4,47.951872,99.176026,205.187005,-29.750080


In [6]:
shapley_table.mean()

A     51.166129
B     99.612449
C    201.211629
D    -29.425385
dtype: float64

The contributions from the MSA are very close to the real contributions.

This was just a very simple example for how to use MSA.